## Libraries

In [45]:
import sys
import os
from requests.auth import HTTPProxyAuth

# Obtener la ruta del directorio actual del notebook
notebook_dir = os.getcwd()

# Añadir el directorio principal 'project' al path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import config
from config import PROXI_USER, PROXI_PWD, EMAIL, SUBJECT, PROXIES
import pandas as pd

import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

from bs4 import BeautifulSoup
from tqdm import tqdm

from requests_ntlm import HttpNtlmAuth


## Data Collection

In [49]:
#si se incluye mas de una fecha o serie, incluir /
fecha = '2024-10-02'
serie = 'SF43718'

url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"+serie+"/datos/"+fecha+"/"+fecha+""

headers = {
    'Bmx-Token': '80f8b40305f8c5067445bd8868c2be2fd29f526c597fbadffa7cca63509a25a7',  # Cambia esto por el header que necesites
    'Content-Type': 'application/json',
}

proxies = PROXIES
auth = HTTPProxyAuth(PROXI_USER, PROXI_PWD)


US\GGI2JU
BoschGil041189+


In [51]:
try:
    response = requests.get(url,  proxies=proxies, auth=auth, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error en Peticio: {response.status_code}")
        print(f"Detalles del error: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    #print(f"Error: {response.status_code}, {response.text}")


Request failed: HTTPSConnectionPool(host='www.banxico.org.mx', port=443): Max retries exceeded with url: /SieAPIRest/service/v1/series/SF43718/datos/2024-10-02/2024-10-02 (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 407 Proxy Authentication Required')))


In [60]:
response['bmx']['series'][0]['datos'][0]['fecha']

'02/10/2024'

In [61]:
response['bmx']['series'][0]['datos'][0]['dato']

'19.3885'